# Sound Filter Design for Robot Odyssey Rewired

1. Game sound effects start as parameters for a set of noise generator functions in 8088 assembly
2. sbt86 translates these functions at build time into C++ code that produces timestamped speaker events
3. These speaker events are queued along with other "output" like graphics drawing and elapsed time
4. Contiguous runs of speaker events are gathered together and rendered into PCM samples
5. The PCM samples are played back using Web Audio

This file is a design tool for the filtering scheme that converts instantaneous impulses into a PCM signal with a reasonable spectrum. The filter would be implemented in stage (4) or (5), either as part of the PCM converter or using Web Audio's IIR filter node.

Contents:
* [Sound Effect Data](#Sound-Effect-Data) captured from step (3)
* [Render to PCM](#Render-to-PCM), generating single-sample impulses with no DC bias
* ~~FIR-Filter design~~
* [IIR Filter](#IIR-Filter) design
* [Sound Test](#Sound-Test) using all filter options

In [ ]:
from scipy import signal
from IPython import display
import numpy as np
import matplotlib.pyplot as plt
import zlib
import base64

# Sound Effect Data

Sound effects are represented in Robot Odyssey Rewired as a list of 32-bit CPU clock timestamps for each speaker impulse event.

To collect sound effects, you can patch the engine and then save the browser console to a file:

```
diff --git a/src/engine/output.cpp b/src/engine/output.cpp
index 3dfc197..c8b3db5 100644
--- a/src/engine/output.cpp
+++ b/src/engine/output.cpp
@@ -116,6 +116,7 @@ void OutputQueue::pushDelay(uint32_t timestamp, OutputDelayType delay_type) {
 }
 
 void OutputQueue::pushSpeakerTimestamp(uint32_t timestamp) {
+    printf("%u\n", timestamp);
     if (items.full()) {
         assert(0 && "Speaker queue is too deep! Infinite loop likely.");
         return;
```

Shorter sound effects can be saved by right clicking the console inspector in Firefox. For longer sound effects, I've been using `--enable-logging --v=1` in Chromium.

The `SpeakerTimestamps` class holds a sound effect in this format, parsed from one of these console logs.
For convenience in this notebook, we can save and load sound effects as compressed base64 strings.

In [ ]:
class SpeakerTimestamps:
    def __init__(self, timestamps=None, from_file=None, from_str=None):
        if timestamps is not None:
            self.timestamps = np.array(timestamps, dtype=np.int32)
        elif from_file:
            self.load_file(from_file)
        elif from_str:
            self.load_str(from_str)

    def load_file(self, console_log_file):
        self.timestamps = np.array([int(line.split()[0]) for line in open(console_log_file)], dtype=np.int32)

    def save_str(self):
        diffs = self.timestamps.copy()
        diffs[1:] -= self.timestamps[:-1]
        result = base64.b64encode(zlib.compress(diffs.tobytes()))
        assert((SpeakerTimestamps(from_str=result).timestamps == self.timestamps).all())
        return result

    def load_str(self, compressed_string):
        self.timestamps = np.cumsum(np.frombuffer(zlib.decompress(base64.b64decode(compressed_string)), dtype=np.int32)) & 0xffffffff

In [ ]:
# A single speaker impulse
sfx_impulse = SpeakerTimestamps([0])

In [ ]:
# A few effects saved manually using the above method
sfx_opening_cutscene = SpeakerTimestamps(from_str='eJzt3T9vHEUYB+Cxkz3/I46NEyuJRLMNzX6A4EiY5pQOCqTU6AoKuJ4e6ToUQeMCIQutUKrraKCO9isgUUCR+sRn4N2rHIFwImzvzu5TPNqprLvZmfm9O969fe+Xn1NznNJqAMqDlIoHKVUZqzdSWtxPaTkw5X5K04cpzTMzO4zPfS/aA1YfxZh7FOcpE+1Yak5jzl+BchJ/azfaZOHscSGvekRe9Yu86h95NV4nP+6kZivaPffnrZhTcVxm4DzmwDSO84ycb0bWx7HKTNvXTVhl6ovNj9MijksAAOBG1bdTFtfCwHC92oh1qIg2AITjk8JeMQAAdMBeMdA1e8UAXPT0adF5jQwAAAAAAFzuj0/23HMCANBjs9vJ83oAANCB9nm93H6TGwCAfio3U1rsRZ0JAB1o37033Y1MAq5VGXXfNObbHIDeKN13BQAAAAAAWfn6s51UbKVUZeY8PnsTVpk6+8C7ZwEAoAvePQt0zbtnAbjo+MReMQAAdMFeMdA1e8UAXGSvGAAAurHeK74VdTkAAAAAMEj1hvtFAQD67PmLbfdPAgBAB9r7J6dRk88BAOB/KjdTWuxFnQkAHSgnKRV3UqoAyNLnPxyloog2XKP1b0pEzbAia6/CdDulOTfq5TeTNI2+nzN45VZcW8U5r+jM+vmnnVjzeGvffjVJRYzhiqzUsfY0MfZXXItZmO7GGs+lnn05UWd2YLYZtUf0f8UbqyPzmr2Y4yM3+z5yP/qi4kqt905inK34V7N2T+KdWL9G6NfnEzXFGypjjSqiz6oRW1/X3ol5M1Cf/nWcijjPFa+pY/w3cf5XI7J+DmM/5v7AvP/dZLR5V8Y4LqIPqgGqY/1q7sbYzdhpPRnVfUF1XC83ce5WA9J+p+lBzLdMNb8Xg1z3Z6GI71dlbF1/HsY4y8RPT4rUHEebt1YexXh9EOedS9Ux1hb3U1pyJcqdyLCHsW7yD+Wj6Jt70eba1HHNvYh+XrJWvhu5fxq5cIPqbfvm9NOzD9WVbc0zxvqw/X+xWu91bV6OpV5b16Y9qFH6rL477PqpPLjeemi2pfbhv/320XBqkHY+5V5LtHujQ6wLyv08s312OPycro/yytl2LF1Vbra/YSAj83H2WF71ibzqF3nVP/JqvORVv8irfpFX/SOvxkte9Yu86hd51T/yarz+BsJM0O4=')
sfx_bump = SpeakerTimestamps(from_str='eJwNw7sNQAAUAMCLkmn8Px1aJHpLGYHZLMAE3iU3Z0+yDFzx7nk7vpYxTg1nzV6xxq3kKEhjkfMDdjwKLQ==')
sfx_pickup = SpeakerTimestamps(from_str='eJxtyDELAXEYx/GncOc4Sqgrk7KhvABegLIYFDNiVVZFeQFWZVWUkViFnbIqq2KnDL71v7ocw6fv83vKCSuTDosMcYCFGhYhkQdaGOBsisRp21TmeAdFCnRNRzghhSY2AeWFkm2Jq6EkubuGsoLfVsTEr+xwQc7WwV53+NhV3THGXROJ2s3SvvZti5hLBTPftxvqyLv0cPT+ivBveP+b4un57wN1ZjUX')
sfx_transporter = SpeakerTimestamps(from_str='eJztXXu0plVZ32fmXGaGufBxmUG5x0WYIECRuwqTyiDQQpQEdFQMDUi8dKCYUENnGgoBBaG4qGM2VtDlRDEFupSivNRaLiAdzTQ7rSUnLYghQLGl1bPXd37r/L7f93v2ewwxXcs/nvW+33vZ+7n+nmfvd7/vd9FjF+1456pSfjvo8ZWlPDe222O7LujRXUv5taBnBH1+l1JOCtq2cykHBV0Y9N2dSrkr6ICgy3ql3B10etC/7FjKK4MuDVoc9KoVpXwg6IigjctLeShot6Bzgh5eVsqJQe8M+vzSUnaN7QWxvT/oOzuU8rzYvje2DwTtH/T6oLuWlPLtoMmga4KmF/dp39i/JLZbgxbN0ilB719Uyj8FvSLo8KC3Bv3NRJ/GYv/siTm6Oeih8T4dGvtXjPfp94L+KmiXWTor6PfHBukbQScQvT3o/tFB6sWx80f79O7Ruf2PBD25cJiOjeNXLxymzwbt06Dzgu5c0KYngtbO0ptp39FvBH1l5HunPeO+t4w8dbojaPQHSC8Kuqn86NE9Hee/FPSKi0v5+i+E311Yyh4XxD2vL2XF+aW85nUR+68t5X2vLuXDrypl8ytL+cw5pbzg7FLe87OlfPDlpdz6slI2vDTi4YxS/vxnSjn19PCd00p5x0tKOe6UUu47uZQ/fXEpJ7+olNUvLGVmTRw/qZS/OzHaf0EpI88v5aLnlfI/xwfmHBf9HFvKs4M2HV3KyqCLnhs4cmQpH31OYEfQqc8u5b/etroceUAprwt6y/6l7BU0tl8p//gTIce+pfxm0Nv3KeWYoJm9Izb2KuXXg44K2nPP8J09Snlj0Jd2L+WPg258ZilnBH38GaW8NOjB3UpZH3Tkbn0crBj4t4F/GwUD/yjwrwRdMIuDwMAzCAcrBi4lHKwY+MiKPg5WDNy+vI+Dn5zFwCuC/nDZIA4CA28L2m9pjoHAwdOC7lwyiIGMg8BA4ODhs1iY4SBjoOIgMPD4BAcVA+eDg4yBexsczDDw6cBBR29+ijj4/cLAHzQO/qhiYBcxBr67AwMvCwx8RDDwksDA2xMM3Dkw8JsdGHj3eavLE4FrvxT0ySNK+dThEfNBKw+L2P6pUt5waCnXHhKx+pOBGasjHoNuObiUew+K2HlW9BP0uweG785i4lWBhzcFHr446FDBxPVBx85i4nlB6wQTnxk0MouJBwd9rQMTa12428pBTKx14cd2HsTE64K+Fhh42k5zdeGHdpwfJta68HOzmKh14W2zdeH5jbrwtFmqteHEknZdyJiI2vDfOzCR8fAKqQsrva0DE+eLh64uBD7+XzFxPng4H0xs4eHCeWDkj+vCHy56OjBx0xtXl0sD814d9FjUfTdG3ffFEwJXovb7WNR+66Lu23xM4EfUfWuOKuXvo/Y7MGq/S58zP2z8auDioUEPH9THxl7QAwf2sfGJwMWt+w9j48FB39hnGBs/HPQne85h46agG3b//mDjd2TMfN1srbhZxsxH0ph5g4yZGRt3We7HzLfRmPkvGtjIY+a1HWPmw82Y+ZB5jpkVG7lWvO9pwEatF//s/6FWfKpj5oqdP8bGHy6aLzZePjtmfuTeUn5lc9QdHwgMubWULb8VMXJjxNANgV/Xh52vjdi8OnD0qoiDoF02RbsbAkPeFX4Y209fEddcHtfH/tcvC5xdX8r4JaX8ddBDby3l4l8u5T+Cn+cHP4fE9rGfj5rlovDN+P2XwdPjwdNhwd9tr4n21kWcB0+fPjdwM/gbi+3PBWa/5KwYY8f23MDwtYHddwWG3xm4fWvg9ruiln3vqTGujXr26Khn91ob/URN+1DUs5+LevYd8fszUdPeHfShqGuvDHz/Vmz3D3y/JrD9hNjuFzh/ROw/Ghj/D7MYvz227w+Mvybo7MD7nw6cXx04/9HA9zWxfTIw/vqgfw6c/1Tg/JVBJ8a5GwLrvxv17+WB9bsGvS/oP6MGPiXw/rygmwPr18bvO2L734H31wbWPxj0q0H/Gu29ITB/SdxzetTEh8b51YH57wz6t8D6dfF7QWD/A4H3d8Sxr8a1R8X+twL3Nwfm7xnb5dHOt+PcBwP7Lw7M3yOOvzww/8m474k4dnxg/u6xfdnKOXog6Lhd+3Rl5IDtO83RmyIPHBb0xV6fNsX+J3aco/vi+o8E7n9hxz7tF/Tx+L33Tv1tpSVBtwTuTwaN9/rbR4P2iv2H49y5S/t0XOz/YmxP2qG/rfQ7cd098fugJX26aukc1d+fDVqxeI5OXNKnuv+J2H4zMP5Zsb/Hiv5+pbNnj38htstr7hifI/f76ppPxgbpifFBwrEtce2R0fZX4vcfjPa32F820d+C6vF6rNLBsX99YPrjsT1gok+Pzx5jWrFzf3tT6OXL0cb6wN/NId/tC+do6Xj/HH7XayrV/fuX9PfPibavi+1Z0c9rRzzV89jeu3Du+P7jfeJjfE89XrePjfaP6e+j4ve6kT5dG8fvWTi3j+PbRwfP47dSPbfv+OAxXFu33Cbv67Gp2X38ru2C+Lr6+87F/T5r+3Vbj7Ec+A2+eqPDcqI9vQ/7tW1uh3lHu2iH21O5eonecA/rKNMX86j8Q369h2VdNTan4+2kC2cPJwPuOXNi0A+UP+aDdbVlYpAnd4/qEP2p/XEdfPLMiWG71HMtvWOL6+FH6hfs48of+7zTP/+eSnjgttXfud0Hl3jfqfssP45vHBn06SnTN8vEMsPvVW/qpyq381MXa5V6o8Pn2K9YD44XtoHGgutX48HZYlp8j33eyaHyYrthkdctfAx9TI0My6cxmfGs1ymO8Hnle2bhIJ6vGevjaeabiu3Mbwuvsthjv633Ty8YbFcxgXUJrNNchPMcC84H1D+nO2zKx2FXlh3bLDc6far/9hI8Vdvit/NHR+qX9dq1Jm8ptjnc1NjUHNPiga8H74zrLr9Xn2Sbcy3i7Ot8sqV3Zxu3X9vqCdaxLrivrlyaxUU9Bt9iW9S2xw3uu7jE9RqP6ieQYeuCYb61hmKakrhUXTAOZ/ia6YjtrDGv14APlpP5Zz/RWkXzbCtm4KdrxnIdOkzmfpRH1q3mW26j9s1tc+7mmlTvhR1a/uZ8fKPo3MUl+umNDsrSwlaODz3v+IJsLqaqTqbMfap31hHrXX0vq3GyGGWsz2pLvq/GM3hyOsH17PNOLq0fVNapkcH2HfZl7Tt/gI3qeMHZiutl9b3pBcPXu1pQ61W1k8uRmc0z/ObziGE39nDyq0+6+kCvr9uVSwdtsG1RziPHlGJoV+xmuO3qNFdnc1s8dmQ/w+/qB65d7VvbRd98vrZbxzFa7ypOs687/+HfrGPlhccoLi+qnVlPXTWE+gL2pwyfZ8rYV2VqYZXDTiWuSyEHxvktuVs83L7I98u6Uvs4H5uUGqXlxyAei7h2szFeF+bh9+T4oL+o7TUGVEcZnjuZWrrXHF/l4rksvo5j3GEI96s2d36D89MLhvvCPZqbarurxgaxw/lPluP5PGp8yAWbc5/av2IOftd7FVPQFrA/85fWXIWrI1SfmpMze2Ocqz6nem+NcxxGZjHU9Ztl4rnRDEMrHrTwU/nR+iDjp/anc2Oufs7GihwbIJdPnP/xcYw3eWzt+GWfAX9ZzLKOOW7h1zzf3Yobp3fOaSyP+o+OJbj/XuI/KuvGxN5cO+jYC7xl8waMP85O9doa11wjZfPc6jOKA85vQC5fsww63lJ/zGrTLAe4HKI50sXzfPxCY5j5wHjE4Y76jM4VKz+rxob9PfNTnhfoqhfgr9qe5qhsfJD5sB6DvOgrw+KuPAbMUB9x/bOfYC4/w1Pmi3nrmdzQGsM6/M3Gh6oH1VvWD/MwTdetlbkcF/v1WPVL4DXX/Bondb2S+uKGJCcpPiu2Op/ge9XuvdFBOwKHtOZBnDksZL9mXWCM5GTgGgzXsMyqD4e56iMZ7m418YLcoXPqakvXPnSgtRI/c2j5UhYfWV3udMh6UD22yOFO1bu2wXHv7IF+M3xpxRfXis5/nY1bunM45MY/qovqnxvlnNbFrtZS3Tj7gp8WbrqYHV+SX8syKj7o9YqFyJHOvzCebvlqK2e36ivEF2OLu26asERtpriJ9nQsze2xL8zM2mpy3MuRzVG4cZ8j5Q99V/4Uk9hX6z7nS9aJzqtwPZzFI+tL53DZL7imaD37ynI4y8ttHL1D7g9aj+gz9q56i7GBc9JW09f0gsE++BzHgca03uNiP9NJS0/QC+oBtYfzHY3xrF/oY7KBxerPfD6b53D+pTHm+GHb9EYHeWSbt7BL22H/QTvof8bMT/DzVbTTlXu5D7SXjbda/qFxiGP6nIvH/xn2ZnpW7OR6stoTtQ30kNWoHBPAIejWzRXwc22tcXjMnvGr83mqY9VZlWVjInfWB+YdwKv6NOpy7cc981EbqC16gkXKH2M36zCLK9WDxgPnBMY6l6vUdzUnMmktoRjYxW9r/RdynNoBeV7jcmah1xvuyeZU2L6aK7J4VXzM5GOcxD7u4fU9LczM6ni2Hdfgin8a85Bn26JhX4H99Zmb4m9L7hb2QK890576v7bj1sYwzSwc9jn1B+a5l/hnht3zib0sNzHv0C10zfUd7nF1ltY7Ls7hY87+es3keH6e73e60/hzMZLdq76tx2p+cHV8dp8++3H9YaxWCc9FVG7oRGPbYUMmp/ODTE6OFeUbfqFzJC5OtG0ekzubZbVDpclxz6vGQasG1zhRvbVsxfsXLPW627BouB+HVTMUb5nuW21wXZC147BQ8VR9wfl0K/c4HWgOdzpVfpGrM1613yz2VPe4b0rkVZ/RsR+fW7l0OO7Ar+byNWPD8ma8OTxcJWsou+aHlAfMOWR6bMUNjmWY2jWGbM0lMOmcOOY72C+0ZsjiUvNwtZXyMTXrX4oLkGO+edSNuzPMBTH+u1jlGG3ZrP7WZ3pqN87bLJfOgTofzXjQeQW9XnWismX+z+d4nWrm43Wfxxgax9m8DvuP1m3qJ7zmLMNe5kvzvKsLnW6UZ+6LMXO+Y/OZRB6NS+2L47xVwyBmNZ/qtsp38rLBNhHbGKdp7chz5mpH5hP+wfWGrktj4vlWHtPPd85E41THme6+LDe6/FhpesFgPw57ZkTnakv2LbYjjzWcbnGfW8eh9kCbGCcwpsEm7LPZc4RJwvYM7xRf9DznCvZhzRsqY+az2L9s+WC/mDdSX8ne48Bvfe/A9Yd5e8VX5yetuVqHpWhn0uiBdeZ4d/Iq/86H2DfULuAlk41lVHmc7rQd9w4UX696cO8auX1XH7rjeo3ag/2AbanvreFezfF6P/PC96mNEIdVXvhYb3TwHO7BGmi2r3uekMmi/LTmstA3y5OtGXRYgDhFn/xshq/T50zKa4aj2hauc/rgNQarxnwfrZoYfewruNiKvyxGtgl/bp1M5peONzemduMChx383DXDDSWunzTPtvKjy3E4ruMg5iPLyV0+ojXbfGyNe26h9zC6fFD54lreYUxX3sR1/G6U2lnXXrTslfkycqe7F/kTtnG1OV+vua9rzt7pTn1Y7Y/2sJZbbenyONvAjUm4z8zvcX19zsvPs7Jrect5wK1/cOtXslq/9X6W4jr7INd9bFvIwDzqXC7ziX2u69Tf1U9dznP6r9dtM/NxjGUur7saKLNv5nt8fCvJ0xsdrr9YZ04Wh9XuGLehvx3/+u0Obdfpmc/pmguds+F9bqce0zV0tS39XgLzxHUbr2lo5VflG/dtmcjvY765H53/cjpzbc4kvsTvALb8SH1DdZvZxvkUz01PJe1ndV2Xn6u/8TjR+SjrIYtj16+O87rm7ZXf21YMHuM1qnydW4emvuR41XZ0HV7FObd+tMrD9nGxpvK25iPcWKQS16zZWE/nADQuGPd5rlFzC/YnDYZnttZj2Ne18DzP4eTPMJvX0GS8OJ1qe5qn9F6OI372m8UOeOO2dazqbOb07fwni1OVQfXAY8ZMN5y7nV+zT2VxDX8GX5PjwzrO/MTpxo0ZnP41FvhbJfqdEbd2SMfkrfmaes1WwwP60xiCP7i86vTBNoDc/D4O36PPllhXkLtVO2S+wDZ2z6+Yt6xW4WNuDY7TgWIW+zI/+3Fx4OYIuJ/WmMn5e5eNmD980yN7h83JCZ54zldtwPy7etPtq2678JC3+q02XovG/CFfsM2c79Stm6urMuv3E9jWGT5kenXzFM5X3JiKeeD3aLrsznKy3evxGcEAHuNxrcY+qXpTnXItxvzxM3fVV1e+wjn33hm/a+N8nteP63nwqfjN/DEuZ3W8q5czDHD5je/NvgUwLbpnQput8cnSHeZ8C8cUBzIb1Hb1ubbGvIstHXOpLrqw3sUWjm2R+QDVh8P4Vixy7Llrs/h27wg4f3FYoT6r9UsL23kuSL9X0NIzjyW0PnD8ZrIpxnPfwE34WFa38v067m3NmbRkVNu0fCzLUy5XO2xRv25ho+ub19jp97EUa7viRGtefmdIaz1eZ8k+B5/id/3c81WOh2zuVvnnNdYtHIQ+s2cqTr8aU1on162rqxQj3LtVme3gqzrvoHUIP5/iWinzLVcftuIzw27+jTly4DiO89ha5d44Mmgr1VXXmNjFCttY1wXpNcyPyzkO75in1nNnbV/fK9P7cK2OPTkG2PdUlyxD3XLucvbXmprtnr0nnPmMXqu86RqCul1rsKjreUjml9ivfOt6Bh3vM9/wWdeffrNC40n1p+dZbtjE2czJV691Yzy1vepD1wdpPLk+s3zOx1lWxuss9+C+bTTHBT5WjQ3z0cq7rTnqlp84W7k1+BqnrEtn2yyHqx/xbzevqd+bVZ54PW6WF1vYhuM8B8t88trAlo9kPtSqGzj/uPEErnPPLVy9qHN76u+t2HC+hC3XkJn/uVhl/GSs1LFvyy9ZN+AH9Yurd9TvlJzdMj2wT6Imc/Li21h6n+rf+b3e565t1UEa95WXmYZtNXaV1wwzmV8+xjHLtcR8MEf14d4vUttldYbzORzTmoJzls6xuFpK68isP8aNqRG/FkfxsaUnzSWt+p75Ax0z2o51fZbmckgW50rQqeJk/a3rXZ0/OH2wf2bzB626Ajpz3+JhfWmNmo0d2cb6vpzzVV6DmI0lWvbkLezIc2hV5+ADMc/f8VS/0HpV5XL+kdWEyJnunROXj1vYy7zyWLkVL3ov+MjWEuBa/f+JDIdVHmc7rTHcO5NsP/ZpJ7vzB50XzPBv7fhgrGnbmi85B7DfuhzndOL8GW1pvLm5dsUCxHimC+5TfZvP30NxgGOMTezXTqfQE9pi/bjvgahfqT6zXOf0ylgys9DjMt6/nI+N3DmWSfXKdnJzTaqrSTPfktW7sHmG84zDThb1n7rltSzu2yKM64p92q6Lf9ynPsW47+7Vdc0tH0BMYryXza9k3z5kn3a+7PCS/Yzfy1XdKZ6oL7p1Phk5nHM20/yxzaxz1LzI/XB+dDGic7uu3m3FJ2LQ8aCxpP6v8aR9rZ/wMrr4G6e1tuxf+mxYa0K2oYu1rP5nUnxX7EXc8LpQnGfsr++MqZyQi+sAF6fOX7IxGvjQb9ZA3ta8qfoY46Jb0+FwgHWC3xpXmT+6OZOsPlUZ2B/cvUqMMa35AWd76NnlVq6Z2Ub12LRgrMqiOVhlczKrfGxv3OP+zymLOxeTkOHLi71f6pga/fKzFzfmYB45fjP9ax7iHIpr3Do89+5Nqx+ds3HPBLP3m/n5QgtXnO0UM5gU61U+59euT2drxWtsa57m+XO+ljGQ6eZlw/zpuvcMO7IxmupK++axp1tP2JXjWK4MH/kdBDf/qnOGmc6d7Cyf86vsGUgrbvl+5DC3/krxC/LULZ7FOVxW0lyk7xV3fbs5s7We4/jgdlrzei7O9NtPej0/z3A5gt/5Vp0qf26u3K3v0mtcXaW6y/KlypL5i7t+DdXnXTrl/RnxJdYN9Kk5E+NVfn/W1fWK+5nfO+x0a85ZnjUyHnH2yORmYvm47uQc2opbXdOjY/gsPqAfHZc7O2gs8bcnOY7quY3Eh/qdq92cPlw9wdfXbZ1rYDu7/wZwuUrvadmq3qPPXjFnmtnS4YOOWTP8UJ3AJroeJONX+8xkZD41nzGf7psB+v+OLnfouz0Zj9w3z0s6Od13u5kXh4uOf2cT9W+n42wM63TbG/Uy8DiA29EYcXGo48N6jb6jU7fu246teqDLRuAT9pme9YsqA6+7wXVZPQa5M2ypxyrv/E0S+HDXWEvt0fpPUYclfC30pN/ZzPSTxa77rrVeq7HO2M3XTtE1WayrjBlGsayqn9a7zU6P0BN432r0ncWj5mXXvj47zfhmHbXsw+1yHIG6vtXqsJt9Jlsn5raQm7+hxtfod9+0NoDM2fjc+T/372oD2Kie4/EX5i3qb2Abvtvq2sSaIz6v/u3yHNrJ/k+Bdc3HFdsyH+mKmRb+O30i/7jnRs4H2KfVrnp9Fr+KX+wLPJ/fmrvMcIzP69wzbMntXzjm21MeK/FcJhPmKFT32dqV7L0nF2+tOkvfyec+3LyZ8zfWucMJ1WuXrznsULtkc3oYn6pfOflUp12Yr/qs7fF7TJjnxTv8qJHdHJjLqaqP1hiX29FY7Lo/+0ZIK8Yz/Mx8HfbZMuGvy3wT32xTeXkNg/LNemj9V2cWk+63qzEyvKi/37Rsjofqf+75El/PsnTl9KxfZ68qq/7fB3wqmwNx3y5y+NmK04wn/i494oH5cX6s4wIXO44frlXdPAiu03eiVO4sJ4NWNfKy+6ZQqy2Xo9QX3D0tX9A28Nt9R57nhrr8L3vm7mRwse3yGLZuTpHxEMT94FuSjvi7b1xD6PfeMt7ccfhGNhbTdXwzxn+dbt0zvlYt5+LO6V9ld/7o5OR73PcfM/24Z0XZ9fqOmT5nVd/h9SxaZ7Zyk8Z59k2MrNZxGMNxo32qTrP/s1PsxzeAMln4Xh5fZfN9Dls5npxtMv/g+Ro+r/91qvpyebx1jNtotdnqg7/TlbWvOnX96DH33Mq1mdnafceGbYV+Km6xTassPMbkmOHnKBiPsh+7+R9tR+fVnM9wznaYkPmAy3eZLTJ7ZzGgduf3s1o+pnUQ96ffWuVrutbAuDFZC/P1vM5PYh4p8xnwVL93yWsU3LpWnf92ta/DAu2PnzVvWzR8DY8B3XuKKjtjEfPa8p9Kuk6Acc35pdqA9Z7lBY2PDGORvxkPXD3X8gP+vX5isG3F7MwXoRdnR71ecT2ze6ZLfb6rbY0nYzjoWfFE9Zn5jY5FurBXv2Hm/BXn3fe8eB6J73Pxq7rtjQ7z7PzH6bveg3ogGzOwz7KfcL3j5kW1P332n8VQPe6+meTacDLpu+Quf2qt5nK0Er+Xw7rmtrJ5VCbnr/rOT8Zr1zyN023d1+83uXh1umitJ+R38HUdCfu1ixXFGcUeneNTe3XVk+ornG/1OVOGoZmtnC00x7i8x32xjty3L1knWlupr2r+cPZUv6pbfYeJ35Pr0i/a2Sjy6bfXFcN0jrLu6/eKFFtUthYmZH7s9Jc9e9L/XHP9ulzpcmZrzUMrZlHzZN+byeR0dnPf5eXrXM52Pt963gd9/C+u+JNe')
sfx_chip_burner = SpeakerTimestamps(from_str='eJztVk1IlGEQfmx3v11tXdcyrSxr/cnfaxclMiEoqENhSUFgB28R6J4Cb9EhkqDoVJ5KIeogQUJQGlmnLkHQf4EnT0FCWh06NMPb8M3Ovt9nxw4eHub3m5l3Zt53d7hlpGalG2juAUaJjnUBNzuBux3AScJ0OzCyB+glXG4DDrcCTYSeFuBjM/CsAIwTugi1u4GlXcBKE3CWsLoTeEl4vwM4SPjQCNwgDBA6twNz24BjhMl8gPUa1mv4n2pAZ4DBBuDSVofXxJ8j9NU7HCHf5i0hfpFukuy9xC9QnkdElzc5FOocPb8ZeFcLZBQdJ/18PsQ30r8i/+ka4G3eUcYg6eeITuRCWkd19ZHPb5KL1aWYpbins8CTnKMClisJ3VRrKu9wh/zHyLZM6KgCHvylX3OOPib7qY3ALfIdJVpTCfRXOcq4nQ35edL/SAPtxE9RfUfzDixPE74EDgmKvTcD5Aj7lZ5l4SeIHyLbauDwIhVS0YuOZean6Jv7SaA67ajgc6qUsp15oWK7QvL1hMPzZMi3pUO6knK2xiC0s45hv2X/A6S/R/yFDQ6azwZ+vZWtLUrH2Ec5PyUdWP5JuEYYrnBUsJAI9a2Boxrfk85HvhO96LQfQ2SdS/vZ7xgn0uX+/RTrTIXDU/rmKumPpx0VvdiYsl78BFq/nCz9jsG6Q4E/ns0tso4rdMb41WedLLEuZhzEznmlJqlP4vMZdXxbj9YVg3I/e3bdF/HRvRDenjMOOubDSkclDvMFqmvR9EnnEBSC8jP5cvnq0r2MmzXrGlKluXyxa5P+WoW3/RGd3Umesz6n3Tddh8/HNwfZMe2vc0hMvXO+HNpX66WHdpd0rLg5av1MzDx998jK+gySR/qmd9jXB5HZV+dfUndtIFU+E9+9jqpTZiy16Dx6F+UeR90d2yP9FjBd9OS1e691LPveODtrOxdds949Xx/kvDILRlDl32PZNTvzKET1Og4cn99aWyPPR/txL/mdtz3TWGtfLfQMpSc6rn7v7T3Ss1709MfO2er0bsnvom+29my+3wZ9Hvv7oO3FoLwfdt98M43bRZ/Nnte+TT4f3y4zLUa8+ew36zl/1C5aP71/vvlG7VFUz/XuiO2N+b/gu7P/srO+N12/q1KT/d8Rdf91bfq9sv3Xtdr+RP02xp1hrbsY9Tunc/PbH3cHovr4BxAdSnE=')

This is another technique to combine with the above patch for collecting all table-driven sound effects from `GAME.EXE`. Based on the "Sound test" from `fun-things-to-do.txt`, this copies a sound's parameters from the effects table into the sound engine's queue of active noises.

```
function soundTest(effect_id) {
  const current_room = ROEngine.getGameMemory().world[0x300];
  const mem = ROEngine.getMemory();
  var queue_slot = mem[0x4b6a];
  if (queue_slot > 0) {
    mem[0x4b6a] = (queue_slot -= 1);
    mem[0x4b42 + queue_slot] = mem[0xd2a7 + effect_id];
    mem[0x4b4a + queue_slot] = mem[0xd2c3 + effect_id];
    mem[0x4b52 + queue_slot] = mem[0xd2df + effect_id];
    mem[0x4b5a + queue_slot] = mem[0xd28b + effect_id];
    mem[0x4b62 + queue_slot] = current_room;
  }
  console.log("soundTest", effect_id);
}
{
  var next_effect = 0;
  const interval = 5000;
  setInterval(() => {
    soundTest(next_effect);
    next_effect = (next_effect + 1) % 28;
  }, interval);
}
```

Must be run while in the main game, not lab or tutorial. Post-process log output with this:

In [ ]:
def soundTestLogToDict(log_file):
    results = {}
    timestamps = []
    for line in open(log_file):
        if line.startswith('soundTest'):
            if timestamps:
                results[this_effect] = SpeakerTimestamps(timestamps).save_str()
            this_effect = int(line.split()[1])
            timestamps = []
        else:
            timestamps.append(int(line))
    return results

def loadSoundTestDict(sound_test_dict):
    return {effect: SpeakerTimestamps(from_str=s) for effect, s in sound_test_dict.items()}

In [ ]:
# Full sound effect table from GAME.EXE collected using the above method
sound_test = loadSoundTestDict({0: b'eJxtyDELAXEYx/GncOc4SiirsulKZl6AsipmxKqsivICrMqqKCOxCjtlVVZhpwy+9b+6jhs+fZ/fc7fy2WxUZIgDUqhhERF5ooUBzqZIkrZNZY5PWKRI13SEEzJoYhNS3ijblrgaSpq7aygrBG0lTILKDhfkbB3sdUeAXdUdYzw0kbhdi/Y1ty0SPyqYBdxuqKPwo4ej/1+Mf8PvbYqXz9sXnLI13A==',
 1: b'eJwNwy0KwgAAgNEvD/F/m1WDCDOJN7CPZdOKBg/gAYYgFs9gsXgAwfXJskHM4gE8gfg9eIfzfpo0Yeervx614OjawzbMvfbJDzc6sPLFH0ddyFw47MHES99NH2au/HIQwtZvLyJIXXocQ+6bf94M4Ok/dFYW8Q==',
 2: b'eJwTELqr62PFwLAQiBdZMjB8tGBg+GTOwOAIxE5mDAwTTRkYAk0YGHyB2M+YgSHGiIGBC4iNDBkYAHPiCiU=',
 3: b'eJw7VG2k38zIwHBnBGNTJgYGj2GCFwLxliGCeZkZGBQGEa4E4p4BxAAVLXxW',
 4: b'eJzjNG83uM7NwMDPycDgwAWhYexAPgaGb+wQrM6JYJdwMDDcZWNgsOeE0MiYDygXDVRzhJWB4SsbAqtxQMRAeAmQfYcVglezMDDYAfm87BAMEgMAmiwWyg==',
 5: b'eJw7fOCW4XxmBoYfdMQANzIdqg==',
 6: b'eJx1U7tKBEEQrL3bhy/EReREQViMjA8xPUwNFDMRwdT4/sDcA/0GMdBgI2MzYwMDA4P9hANTA7vZK6anGYNiempruqt7Zl/fx+OTPeB2BzjbBk5HwMcWcCjY3wB2N4GvNaBZB55WgIngbhm4kf1lBfxK/F0ALzlwIOvDEPhcAp6HPS6Eu8563A+AH9FNBDOJr7IYyilaiZuyj98kR7fQaux187zndV/nsU4xLQPHc7pvXV2uqmuzeK85bE7lpwt/8zzw3SDUsDn/q+fBOorjItYfrabnpTXPq8DVbra+J4LzoG/2x1XzjIqgY2/sl3NNebLz8Jye5z3Rt/U2N/7rPPZr69CznyXfxcz4pf+mTOtSb9D6Up/M4TX01rkebQ0/b4s2i98Qc1kudcb+C/bbYxV4fUPep38jvhfes2rpIXWP9v7/AN0HjFM=',
 7: b'eJwt0u9HHAAcx/HP2FE0ikWxsXTXanW12651t3Vbt/XrWrdqFovFouhYdFws2oMoio1GUWw0iqJUNIrGRqNRFEXZomgUi2J7EMXe9H3w+gve762fvd6pdCnOea4BMbOCHfhdUqUZxZxJypCc5g36zAFOTe11qRnfsG7cmVKRGcKEcWRJKYiiy2zjCKEb0gvzGUsmLVvy4h2GzQkScqQmvMYa9kzALVVjHAsmOVfKRCf6cQjlnavDK3zHJjw3pWJ8xLSJ90hX0IYe7OIvwrekl5jHsnHdlnx4jxGcIdErRdCBDewjmC89wyS+IvWOlINuDOIYFwukerTiB34h3yeV4xNmcckvXUM73uI3au5KjfiCVWTdkwoxgDFcKJQuowVb+IOSgPQcM1jE1ftSLz7gH+Ie8A5iWIG/iF8wijkkBXkEfTjAKWof8gXW4X7EC5iAo5gPEMU2jhAqoTuWkFZKbwzjBAlltMYa9hAopy8WkByiK/pxiLoKWmITnsd0xDTiK+mHHuwiHKYbluF6Qi+M4AyRKjphH8FqGiG1hj7oxjHqn9IF/wEzGp8C',
 8: b'eJxV0klIFlAUhuHXhWAOqaXlmFpaalpaWlpaVpZaWlYaCBZNgoILFwoGBQkKbgIFAwmEhISiBIOoXAgFRQUFRbUICrIBCgwsEBKaXvgFbfHw3QN3cc85NzxhsrC6Hob1pA6+H4Yfh2CHWg5CbS38OQA16tgPjTUQqoZq6Nrnvb1QqqYqyKiENIVUwO09EKypcrikq7vgiE7vhGLNlsEtnVTbdtit4G0QpPslcFbZKtoKI1t8n3qLYaYIxnRMrZvhzSYYUJB+F8JEAXTq80Y4owJ92ADT+XBUcerJgzKNrodmvV5nr0pXfy5UazzHnhWijrWQp6FsyNdUlv0rRl2ZEKvna5yD+lbDTzVpNsMelab2dHirKr1bBYN6sNKezXtmrs6nwRfV62sqnNI5RWskBVJUrGcroEInNJMMlRpTstqS4KZaNZjofJSjAQWpLgEuKlJZmoiHKHXqSpxzU5Eu6+lyiDCPm8NKVKF6l/lXlKRujcX6b1SuHv2NgXiz1RwVatbAUmetzDkt+rjEval2Tr8+RbsDszM6YFzhqtGjKN+mSRXM6dDjSHdkNkYGDGl6cUC+5+tmtx4qboEG3YiAl/qmsjldehU+r8S6OTzgmn6FzSu17jPvhAXyhdIXaNLd0Hmp1tWh/7ug94vmpVm3m/8A+PeQ6g==',
 9: b'eJyNXWmIZUcZ/Xqmu2cxk6Td2g2XuAc1btFEXGKicVBBjYoLBGMU971FExUE0yrGJKLiMhLEpQXRH61io39ccAE31PwYEWKwBTOioga3IOJSxZvDnHfeOXX7R3Hvu7du1bdvVfe+r1x77qO/+6mqg9dX/edY1dbHqh7Ujhd8uOrcD1b96rqqG66puuLqqp+19uT3Vr36PVW/eVfVeVdV/fQdVS9s7SWtveuKqpveUnWitX+/ser81t73uqqLX1+17w1VP3pV1X1eWbXdfj/s5VWvf1nVdZdX/f6yqoteVPXndn7epVWH2u+PPL/qs8+r+n47/uC5VU94dtX/ntXGfmbV359RdXNr/2nt8tbe+tSqY0+ruvvRqpWnVN3u4qrnPqlquR3/2I7/uqjq6e38hgur3ntB1dVPbvcfX1Xt918fU3VNa5c/turb51X9srXrHlX1gkdXXX9u1bWtXfSIqtMfWXX2wxt8D6t62zlVD2zHMx9adfTBVbc+pOrj7fcV7fz7D2pwPbDq7We3Mdvxx/ev+nQ7fuV+VQ9t58fuW7Xezr/cjue048fvU/W4dv7Oe1f95KxG83tV/bYd73bPqk+0tv8eVZ9v7dft/ivu3nC4W9VL2/Uvt2tfv2vVD9vx5+14a2tfave/dpeGd2uvvVPVJ+/c8G/H37Xfz1lvY7Xfr1mftVe0Mc66w6x9qrXr27VDt696TDv/emu33PZUW2r3zm/3/rvW6NZ+/6Udv3nmrH3ydlW/OHPWPnfGrL279bnm9KpvtPM/t+svbs8fb+cbR6o+0Po/u1174Wmz9v4jp85/2doTbzNrK63Pm07+/lZrDzg8ay84eX71abPj+W2eC9px+YzZ8VGHqs5o7TPt/rXt+M8Dp9r9D51q/febDza+Srtpdf68t2va+fdWqu7Xjn9o4/6jXXveodk1tH7twtYe3/p8cbnpzcrseKTN89FDs3M09PlTu/7ddvzQ/ibTK/PH3nBPGz/Tz1ca3lfuq/rC/tmxt9NWZ8cbl+eP2k8b+r2uwXhrO36wtcuW5o9/Wz71u7f3HJy/z/d6Q/9+TH10nt6+s//UtTPa+SWNlpcuzdp17dq39uffem1TfvfzNAZf0/tbBxbn4PMRXNzX9bllOY97r9X5eTHGNs3rxk90STgrvJ3muN8b7h87MuvT4frqoflx0Qcw9zH6Od8HjBgTtL3E0Jfnx/Wdk8/cfDjjpWMp79cMvZVefN5xGNGSr4GXDL/j0fGDfqz+PMuDwpKuORrr9asOnqJPP164soi/jgOcesPzPD7kl59XOmwvLeLEODsZ7/dPDGjOMjnin5Oh3nZPwskynHTP0ZL741zlTvUaepvkx9kOzA89UjxA+8T3Ee36s7uBbkmeeLxOO0cTwKo8T7TbXlqEjZ9hG3LC6BPbX7WvijvzSPm1ZuBV3HVMlnP0uerg/PNKh2TvE7+0rZ3UR7a1GBO6PZIB0Jj5pDYSfXcJT5UBpp+jg7NbU3RQnmj/bRnX2Y6dfZ5+TgdY31QP+nxJp9eWvVzofAyf+gP3jMN/5Jswnsqujs32kMd0+gvcnf6qTXQw7cqYTm94LNjgpGNMQ/ixJN/wa+BV/72zb3HsvZwrzXBkm6ttJA/bS4vXdL4e14D2iMEUBva5aMcDTXgO8BPjj2B2fsT5Ah2X++8Oxh/Jhs7dacK6pDbOPb9LcF15YBoflnc+Tzqu1/meiy+TDrgxYQPSHGqrHT5sM+Bj2Yaz72DeKQ+TrCr9OR/AdcQWaqPVBigOLtZQPicaXEp4og/kcJP6dBuSxgD/UyyltsD5T5WFNF6aO8mLkz/FRfv28bpNdD7c2TbHX0fn5Ku1L8+pOYeTCR2LfY7aPfRx8YfKNv8+sX/eHid/zOM6+Usy4fytwsD+FnKa9I3nYPnGfcDZdTDlhp2OjA/OXQ6g/HNy43y52qAEB8dB8NfMa4e/y6WVFylWH+XvgAe1IhdLqU1RGWQddPKQ9MrBjVrTlH6ozjr4nC45HND6WCkPcvTD3B3/lOM7u8Py5/QMz6aYSWmGZzU2VNz6tRRj6/kJkSvk785HOj/mZFXtiIMhjc1xD/RWZdyNr7E84z/K0dw91mH+vWZknO2NPjOal20I+mEspjPbLa09aQyU4GddcDq6s2/Rlm1J3uts9Ag3xyOVH+ZT8sGYJ+nrSPZG8Q3Dz7mC66vPwMawTWfeuBx7RLtkU6bg6dd73oB4A/ozqo9xHoBnICcnBs+preE5FQcHd5IXXffQsaZo5ejmYgzIX5/P1Syd/2caq/4Bbo0reEzVb5YP9VOMm9rykSw6fNVuMexTtHbX0TaXFnFy826Y+kmSadX3pxzxMoi8S20R48p15+R3Uoyi+Dq5dL9TzJhqRfyM5jfO/rFsuOvJ/rOcsfyxrKSYKukW/BPXnFTWR2NoPOzqD45OylPwK8mzky3VeVdndTR0v52vSDCjrqPjIM5R2WCeJjl3ssDzK/4cL6T1iu0lv6Y4qhEpTXuDjUi2a8qn4dmtA4t5Ettpx3umta4VjGRsV3jH9GIdwjNsizh2U7ycnCBmUFhcvD6KKZze7KWmpEeGt8vAHU/z95BzMI241uR8l+Nx8gnsR53cqzzqGCo/2if5LNVJ8Mjpo9NTxoHhYzlkHBSfkb0FT7S/5ukKG/sSXjtzNsPpI65z7uXk2/EV99dX5mmheDsZTzRA05qz2gLFodOB4WbaJXsNXNkuqHy7PMbhw3M6/RzhyzTpcrkmcsW855xR53O0TbLN8zI/cb4tz7q4mc9vDGte3HTdS8dzsG9Jbs+0djROdE/64+o0zhfxb96rpLAn3wxd7ddhe3RNzNGbYyeMMSVfrsab9l05mug12AeXS43ketP0Vb+OI/Iuht3VDZm2GJPtHc/jZJ9lh/mjfgRrDCN9cnRyYzs66TpL8u/ATWPHJH8sI87uMY3dXjaHK+vF2vKiHvd25YExrZT+andSDOliwN4QWyHmG8GfZEH9A+s+8h+Mvb3keap0d3EL9HxKjtQWMmwbQvNRvYvp4dbGnQ1X3XdjcjzLvEz7U5zs6Ziuxqr8VL+uNNQ9aIoTx1LgsfZRWe993Z7M/pzunVVdHtmJVINJMsx9mVYbq3m8kU47vnR6jGI+lUXWmy4TPNZRGSfloMzXEZ4j+vG57s3YS2M69VwIsgc9VlpqfM/+YRTfMw3Rj+di/jufpDrkcpbEc3dPdUzr8krzUZwA38R2YLR2nXgAfrp1R5YLpQl+a77g9GnkMxMPnF6hceyW6Mc20/HI6cdoz2CKv5P+jvwi5Ahy6er6upeNYU553ghfvdef+/GBef5yjIxxNU5Wf+bWZJn+LldPNHJ2ieUgya72GdUg1pbn+8I3qfyrPdD1x6m5mF5rFCdrXqvxRPJHae07xZgYp9/fJHy01sHno9gG4+o6XpJ5xcnFWWw7Ru/mJFnnfi6uYP/Zm9tnvD0xZ+Ip+vK7DCqHGv/hfopXmEdOxx3+CT6mQ4rlYffhQ5wOJ3/nct2kE2xLtK/aE+gi1g1crgI/rjgxr1XPHO2cvuF8FNewLjNsauuSTQLsKXbp11YPL8ov4+Tqryqb7n7SqRP753mC67s0ptZEmQb6roLyWGHuY7hn0LT+MlUbSnqhMHAOwjEg2zT4rql6gtKF6cw14RSrpHikX1cb4JqL91wMM9obwee81pBiCKYR9Fn5qnRzsOOdvvRs4u+WxCduH8PIjyrv9Jq+H6V2AmPqexuu9fu81xzwJnupcqv07edYw3H0ZXg0Z0q0cDaoX1sXHXA6nN752iutmR7OziuNkr3EWqabR59TWiO+ZZ1lvqm+9rl0LVN56vIW9RGqY8l39sZwsfy5cRyd+vMap3D/ddpnpPYryY3bB4J6m8NFdVvHVf1wco1rukbi6MzPqS5fKrCr35yyXRiz05TrQa6uqThwzcfJl6MN0xfy5+yug3kUw7Dura8swsKy53wMv9OntOFnHTzJD7LuufdKEn90f2+/13NorrnofnHtz3Kv8uH4haZ7wxyczFvO7dP6bZI5nR+8SO8KjeQ/xTf9nPehuX0tvPfD2QVHD7YH2Oed7LnLH1OfJP/pffE0r+M5zt0zI1s+sh0bq4tzJl+tNirxdYQL48148DmvfWu8ynZb763TelDSkeTbVE/0nZ6r5NsGgDflPm5M1wfvmytcGyZndLLg7LrKg+Njkgelez92enKsp77Frd8ofTU/TTCpz+HaXYq5dKyj5h3wEX9c/SC9R+/qH44nypspezrylRwX8ryoS6RvOqifcrqtdmMkq0lfeuO1aYaJ5dG9T8J9WMadPLh4FHaA65ro19fu+vmO8DXVs0Y1kN5UD1RfEt1d3q22L8lnymPcNww0plc/MZUjJlvpamW85gLa6Fgsj67uAJo7XQL/Euy6/qoyyc9x/DDa553WD5KvZXlLNTq1hY4Wyv9Uo0n0VdwBM8dhXN9wcjCa19kevudsYp9rYzXDr/ru6DGysYk+miMkHwz96Wu+OzQn56ZOF3j/gNNdp8P9qLGo7jFJMpxsAL8Ty/d5z0p69zbVeJjPLONprc/Rx/EafbHnXG0uzpm2Lk5Q36A+kufUuBbzbBq8RzgojV1sqbqi9Hayv7a8yGOWe1czc+uLSQ+Yn45mupcPzdVmVK9HfJjCHfIJ3rs1L421XaypdEk2jeEc0Y9t5ebSIv2cHDIsCrOjh4PfxUq9cV1J/bir3SW9SzxxdgCw4IgcSN+pcL4Gz+N9O87VVXeOm2+uKa4Y1+WiLl4f0df13V6a35vqaAId1NiQayeMH+A9LnuHnD5DF5w8JpvkYmHo09ryPI6Mi9autNaSZBX1QbdnG0dXb+L1eWcjE44jG8J2WPdl9rw/vU+iNMQYvNfC+Rm1KRwD93vqz12MpDKYjkojblMxu2t45tiR+fH7Oep9CoOuMSQbPwXPFtFJ38lnfKbo5Hyqo8kJGnd7yY+d5kx2nX/r3ieN/0by5nICtWsjPB1NHI1Sn+2l/GzSLYzJtWpHN2d7k61VWqidVR+tOaHyzn0PQ2UJtcDe131LgOfTuMvNqf7ejZVw1xpykg2sWaVxXK7F/RzPRjxhnzGSvTS/0p/Xqx0eeHfA6STo6+zfXnwV6NfnuNK819qfTd83dLG9rv2N7B7DxfBq7t/Pubat8uT2ETn5cnxOflb5ps2988fxpov5GBbAq/tX9Fzx4vxHx3K0ZPng+67Gqe8kK0yMv9pI5v8oX3D2Qe2s4ufsTL++G/ol+ic96NdZ724x+j3SJ8RziLFUBlTu1Fc6e6bnI/hdY31BPZDn5z3CPAd/20TpqfE8y0FveK+cnxt9g1ZpCB11dXnArt8TZVnS/C7pbrrOvk1/O5sD/FN8lHB2/hy0ZVuGmoL7poG+R+l0nsdVG9DPvyDvT/C51hI5dnd+O/F2lFuonXFjTfkxtY2cd7JcsK0d8UN1b/QOs+o/YHff7NT5XNzAObyLvdHPvbfn9juOdE79BmQENV21Czo/xlk9PMbVxeuubjvKb5g2aRyM5fLLhLuL58E/fc+NbR/oo9/Z5W8msQ1zPOQj50S6rq1N17dVZpVGaSzdK868dnae6dn1iOsGuifBxSOjsXFd1xBQr0DNVW2+o6XC2tcpWF4wh8aX+j7aiDYjewS9YfnReEhtcW+uDsO8xNzQvc2lxbk1bh6tWSf70H9rbYp55uTKvZMxpc9OXtW+uX1ILufWGGnkj5XvzNcRTViPt2X+9B8j+vzIBmgfrp84Guo6DY+XcOnX11cyPVKsOfJ/zoZukM3nmqPyzdkFJ3ccm/cj1yY499KcJvG+X9PvU3P8qTQFXdw3zF28kubX34hJXDygNFOebhPc+t2co4OYhXHVuabqNHqEfvbn9PtdgJtjFbbDe8kNdR9/WpdlOunYHMfAPqZ1Y/fexYbA4PQr+Q2nQ+o/2Ec4+HvDng0Xi+q3KJ09TbhhXsXPxVX9OPUtYJVbjVN5HsWR8U6yATnnd2+TjCrNcc29w+Hkz+k530M+pLrv8OK5R7ZU4cC8uvbAcG2Y+Lofd0QGNBYZzedkxMl9iq3ZPiSeq0zgyOtKvObt+Oxy01QLdHBCn9RGOdjSOPA7u8Z3pHjO4aI+Tm1fWvfAOb/PskmyyLjxtxmc3nKdys0Dv5PkR/nk3gFwNFZ9c7+RFzH820vzNtHptOquwqD6m+Z31/VZrCfrWG5ddySX6Zzje40X2Pb21uHQHCHRmsd0/6fCttn5ztF7RvztZ9DA2Q1da98rP5SnjJOrPfJ/n7CMOlj2Ih/JXzAuTLfe373n4sbgOdQ+cKwAmFXO0B99ujy4mnnKiZhnCu8ov2OdS/aKZQP39mK3ewNfuZ6nvsX5OdBM5+D/ZVGdcnLg8Ha5U9I3fp73I/Z+4I/6Pperak1E6afzcn2F3xXn/zEY8QHXOGdxMjd6D4bxcN+x4HFVhrUOrmstzl8BH/eu7xWnL8Ko39JjmUu8dDYC33ZKuQLoOLIBCY4p3wT/j/n0O1MpT0lxf9In2E+nG+pjGfe9/KeI02v+vjxwYL64tTeFxa3da1Ncp2LnkZ3Acyy7Oq++g6E+BdewNsb6hyPXdTh+1hg8yZvOi2/Cst9Xn6n6PvoOrrNj6Mf8Sb5XYXf+131TzOXbzM/eVKcYVpcX4Te/S5F8uNKd5dXFDjr3+srivLAb+p9VfO7WbqfoqmseGuuN6jbqu5T2Ka5gPnE/lqFRbsp0d989du+R6D55hsv5bzdfoh3/9w368/s4Op/b64ffO/sW5Uf7Me0492Daouna1Ag3RwOOR1w9Ka2hONuT8lCmDcc3LqZwz+u1LVODUn1THJ1OJr6rniVfyU2/UY3m9qAoDZX37PPZr6W1W+Z/tx+6Jr4XPHWOJEtu37bqGM/pvg2tfdRmMizQJ/cuk8a1vR/2m2NcjKP7BdycGu8nH9Ibv/vo8NYxE77Ma+aL+pLk6zkW5jmRg3A/xBpMW/TRWJhroZyPMgxKf0cH5p3uXxnpPP4Dg2Ur+W5u7t10J+/JtnBL/x/Zj/zf6KDl0dVFXqleO1vq6Dr1X5E8DtMG8LBNV1uozybddHtEVV6c7jhZVT1yOqD2TNcDdb4kdzpmqmVp7S7VGnGevqnq+O3m603/c9DRw9FO3/OAfCoMbn6lPXjl1vhVtpJuODr289XDi/A7+8Fwj+wAz6H1c9WZfk+/AeVi0JTL6nhTeDvZ5XrMlqxLpXwJ55rjcx0m2X9HJ6V7ys2dPXWyMfI9jiZOR3tf3ucE+FimNadOc6q+gj5OD3eJL7iH8xTruHjR6QlfS/Ki19P3d91cup8Laz0uvmV/pPPzu+fsI5LcJ9lS/6VywL8Zzj4/71cGHG5PpearyTcxzdmO6LcgNJbH+ei/rUc8VFlL8cPxg6fiTlxbp5rdlH1R/oz+L8GNCf+tvFN+Jt+lsunsp+bhKgsaF0Af05z6/XA39qie6fSzX3d7hHlvr8v39iIX3J9zDYZd9yQxD9z3pnC+ueRlzPlMpjHnOg6npFv8/W03r3u3wPF8xJ9UH9iQ2MbFVon+jh+whywPyH/cWLwXVXUv6UDyDSq/KsNKH66zOnx29i3OCR6zLVdeuFhW6e5sO9fb0/8zooHGSV8479OYS59z9auRfoxi7nQv5dMux5mKt7ZCzZd/sx9UXdR3aXmMVBfrOur+W8Z9N2VEEze29utHriH0uIXf49E4gHVb90mrfdDYhZ8dfTso5ZiOn6P7Km/Jzrj3+dXGs96M5nOxjJMd1bF+/zzKA5Quae0w5TvJniZf62SkH3kPiNbCMZ/7vlTSQeCn+udsNfB2+zD0OdUDrLU5OmMcly+53NXZDqdf6vfw2629KPy9pqT7O9z3CdzvKd+jcYTirDjgOygcY2js6XIB9ekubnTzMm+3DizyMtFZ8Xd+WfUxxTQj2rA8Kg87ju4daudbHL3xbB+Hv/mtcyqc0Lsp/+hiBrVjCpfSjfeYuPqRwyvZKuSDgIX/Ow7P9HPnd1X3HY0R17gY0OkQy4TavJEvceMlPrC/dLqT9oEjn0/f33L0cbRPObTr42qPLKcJN9XDJAvOP7pvWzidTf+LqXgzHGzDHKwp93L0Vr8xkn9+dlRLwDnvnf4/NAMm1Q==',
 10: b'eJzVyj1IglEYhuHHzud/PwQJNQQ1R1CbkY1CQquoY2AohFDgGtQYFDQ4FNQo1Cjq6CDUKOQoGLUKthXk1h0n+AyKCloaLu7nPZxwyUTNhHSBZdyMS9PI4HFMyuEAEeRHpSus4SQstbGF+5A0wAaO8RCU5mmR1hFAAucB6Q4pLGEX137JS9N+6wx9n7XI3vdZTUxhFUlceq0eYkP2cOu4JrmzjquMF/PRCm9Hxqoad7cw94lN1Ea+9oz1bxyi6/mZWf7ueH6vAufdwtD+C3Gc6v/ovLUgNbalp7w0k5NeAReWX+o=',
 11: b'eJwNzT0OwQAAhuE3sUnEf1Gt+ldTZ4s4gsEgYjZgJWZhNZlFHMAsPUI3HRzAASzEICK+4ZmfdzDoROKwk49MEhBKNwlXMVLQl7U8ZJQGJwMHiWZhKXdZGODLMAc/mebBK8BZTBMusi3CV04W9GyIlWAjYwfaZTjKqqKvCnYN9jKvw6wBVhNe8mzBzYU/yjkczw==',
 12: b'eJzb/2G11X5eBoZIPgaGt0C6EYiv8DAwiALplUD6DzcDAwC6QwgO',
 13: b'eJxbc/qf9QsPBoZQTwQGAFMJBbs=',
 14: b'eJwNxS0SAQEYANB3BdHfYv3u2hOYUcwIomJUwQHILqBLuuESguoESJIT0H3hzXufl+PvkLRgE29zjhmXAYtw6rPuMQr7LrMOSSjavFJuLXYhD6UmnwbfhFX41bmHR41peFY5hEnIKlzLzMMfV0UWlA==',
 15: b'eJyNVjuLU1EQHm9ucpO4u9mo61qI4Att7KxcFLWyE4KyIgj2KqKCsLURCyXg/gEb10KRNDYLroL6C3ygoEKa7ZQN+Cyd4WQ4352cCRYfM2fOnHndmUk+3ewfOzNPdHsHkdC3jMuMhe0Bq4xTO4n2zAX85fORuYg724g6TNf5/XBLwAOWfWwH1LcG+pTpi1mijXaA8FdYd6U1jr18t8b03kykggWWH+K3D5m/MR3RZdl31js/FfCc73ex7BLrX58KeM+yE5sjRDZkHGwSnWZ8mwk84hHjGuu2GgHCHweZ8EI3+Py7IDrQCFRxrlk+r/D9TJ3oa62MvBiXHW4FusZvfjFdZPqmGiBn5OVucaT3gemTnOjqiH6pBghvofLpIugL7jK/XCH6yXev8wiR7S8CL3S1Ur4TfaGqtzy6n2V8ZixlEY8r5bOVCa9vVH608HWtHXz/h3FxE9FZfn8/C5DzjzxQwatKlOtZeQurq/y+2vi9+tCzULm39iW2d4wLzPeyQAUvKxFyHubx7lmjrJvC7lrQESwVUa4y1JWz+kGZ0kFW1hFe7FufGq/1oe86xbh9G5ONA33qGWPDeNpco1v1aE/8Sd28fNE2Qt7ZOFLv9ZtorbUmfRNbisfvhN/X6thcvbpIzLYeXaiX1gF12rnvQ/TlrdTT5q91tbFgD3h183LH74q6g6ysj/la29ijGEOqH/+nztbGpB7WerTzclyTapHqY7Wl/HozPaep/LGGqfznq1Gn78yhrbPdP1jnVD+jX+yRVL6TgLFID56sjs+e7Ts7SxjHJN+p/SC27D5TyP7VOfdix/2os5mql+3JlC3NR3W8/k39VmDeci/z1E34l3i1Tva93TVenLovND61qbVIzajOt/2WuKO9HsNdb3vM7n2sobcXvPnEnAdZup9wZvQ3BO3oG28/IU19X62r17MyH3Yn98y39naR9Yc1tO+8vZuy6c2+7Setq/rE31bMydu9Xk723DH/s3rGp/A62yofwPf1fHk5aU9gPqmaWNt4l7Jr73FmcZ68mP4BknnNuA==',
 16: b'eJyNWkurW1UUXklzc9PbZ1Bb8VG9rWBRB+L7VZAWiooTLUqLVhQU7KAODBQVhOLNrFDQmVO5glQ0IIoVUQf2D6jEBz7IpHWgSEF0VMS9iR/3y5dvnTrY7JNz9ll77bW+9Tz59XTn/leOR/z+asSbxyKefDHiq6MRR49EfPBsxOfPRGx6KuL0oYj9j0fsORjxzSMRzz0c8V4Zr+2POLMv4o69EW/tifjj3oj374l47K6IfbdGvHNLxLGbI+6+LeLTGyNuuCniwd1lzfUR/1wXcXZXxENlXNgZ8cO1hYdrIo7vKPSuirj6yojby/jt8oivt0d8si3iQBknrij8XBbxwiUR3/UjPr404r4yv70lYtfWiM/KvFTGYFPEzjKfKPP3GyMWyrOVMnYvRfTKvS3rI17aHHGw/P57MeLbMm/uRfzcnY4nyr0zC2XfDRF/ld97y/hpYW2824nYUZ59WeY31k0HrreW+eV2xKl10xnXP3am41B5/+lWxJ+d6VzH6+35gWc86v1fynw4GSfL8/OF7hdlv0cXp78n7emz+rveP9leW19/Yx50Z5/Va9DDNZ7zuoncGxI/y93Zvev1qDVPpz6r++C9laKLvQv5OXHGw2Yv7AF6eiZ3zXTr3O+s8aX7sGzqqPuOWvOy1bW4xro6Q3Z1X+ytZ3NyY17deRzPrC9+jveWu2t84Rl0wGdoos14qutHpBfQOEcyVZqj/7Cy3PWyhD4mJH+8W9/LeAIvzPt50bHbj2UGewJvrEusYTnxPk5XWDdKdKa4UZvBPcZ6tZuMVqZP2LjTY70/aXtaelZnE85fuPOd6s3aA+9faUHOTKffWduryQYuJmvGUObj6nWf7JIxDVzgN/sCzOz/6plGtC/8q8ofeyiueL3qmX9P2rP4zM5V6XWXvJ2DB7briiHFAGMI10OSt9ONOzPz3+/M8sN61rOy/kat/LyKd46RLjY0+XToAnxqDAO/sAXnV5p0U9+7c8P8esR2taNBN7d7XgceOQ5kcRE60bjIvsb5AgxgndcNxeacTOrZ+xQPFWdNcdzpX/MM6KbfmY0DmU8Ev4x9xzPvp3FKMas+l3nI8KK2CN+CfVWnvFZ1qvaFcXYp1+vq4vy5nQ7Uv64uzq8dtmZlBQwrPfYrbHdONpy3OSwyHdDqJ3pweuFnHPdZrvX+h+vX9DJpr8Uw1V0d4563B8U6niumWH5Ko76PmO9s3ekK+2pen10z9ist5HzKIwbrHbQ013e5u7M5vVfpqKxVh+Pe7PssF+xbaWj8zHwK62gg8VLt0L0PnbJfV9lCrohXDhtMT3MvxiZojnvz8nRxTHkdSBxu8gP8u+b98APqB9W+nP/jPFB9QBb/9Ays4yZfnl2Dd/UzvE51BH04PTXJjPUFX+3qXrUVzvGdDPXdrL7U91ners5rslP2UZwv1/e2E78uz1cbZl/BMnYxwdmZo6n4VV+9SjUpZIb31E9W389YyGzG4cfFvUp/pZe/zzJg/Wf+vklH9d2VnscB1yqub8HvuNycedP1w//JJ3KqOmutW/XDtZWLkY4nnEPzgMyn8Xk5r3F6YTxjz36CV16b+SXtYfAzrSVdTsW45n2AccY8cA6a0J2LUQ4rdUaM4Rx+2JrHjPqezMdpXM1sWmWovU/1Wah1ue8EnjlPdvs7PY5a/lkTrhyGqk4gb/Y7fWNz2zbO7zfuNdeEbL/aF8xsWWsKlT/nflqrfdSel73LifQe4tY4qTkzXWAv1xtkv9nUd3B4yrDImHR6RmzQXvVy19PJrsfi+zQGqh/RmjizM43VLEOVQd1rpefpQo6up615G/jbnuRjjk/mVfOdLGeH3BFDsvNrnol4x1jM6k/mS+OJ9rSz3ovrRWX1kLMd9IPZbrJ4wnuynrTvV4fW8E1xj/ltypW1LuF+EWx22MrP4PABP+bODpngnvbDnZ/P8Ke9ZLVd3p/7qywn7hH2xXaVJtvQmHI09rG4Vt/UNzaX1aigM2nnzx3edR3XV1rjOZr8/dHtx30Dztcye2dZqL51XXY2xmNTndHkCxQvtb+m/YmKda3H2T54T5w7qwlYH1oXON038c71HeoC9eUc8/Qe6Lh+NceULEdh/s9RvnEx/+vq7AHVb03x3cXgrMaDDrRvqf5afQLwADpN8Ypxm8nJxQV+tro4e8/ZBmPL2ZHjj/XAceti+HKxuvLIOS7jv15f6M37b+aNY6f6tqzXzms0j83is5Mv9AN9ZTLjmXMy3D9r4gRkA/onjZxdLGf5as3G73Le7755OFsCfut99gvsn9VmHEYxVzuoM2z0ge68rJ2eVa9N9Rb/dv0qPiv7XJa9w2/TGeFztaei72AdxnI3p6mYxXr3HZnlp/mF6lVjkuYA2jtU+pnM3ffiLJ/nfqn2GJnm0OyvfUyVh9q78uRqpMyfs70wltQ+M5nwGv4vQIYn2HD9HsxruZfaXZrFL/jhbxJMz/2uukJPwH37r3O/My87HpqvsL84LLLi34il2gtkDGb/U2mKNxPDK/Z3/ylgfWQ+XHttg+5svs96ZDp9im+Idc9vnD6rutR6Gms5vuh3d8xcb7OM9YwOd3pO5l9zPI0JWFP5OiI5JueKjDHokus0tRHXW3V1mYsPrHeHjRpnmU+uOZ1MdD/2ay5XUR3iGzDjmOWS+fiRWeu+4WQ9q8zelLfsHfdtjvEDuufWzeqC6XWX5ulznOP9BsJjdp6mWAo7Y34RfxXb7hugwzfzgjw180OMAf0Oy2fW8yOeK67dOXUfzocye8p0zvyzvpVvnEdxXOd/AV3+3eg=',
 17: b'eJy90kkohGEAxvH/MMMY+zbINrJGKCnJUEOyHMTBQZIlHDhYDm7KclIoJ0viIGU5KUuOihvCTaEcLbkpUfLUR8hSCodfz9d7eOv9+psfE1174VAhG2GQZYf+UBgIgaVgOAqC+0A4D4BTOfOHHT+Ykn5fCJVLHzj0hhZxyIgN4mTNC8rk1Apdsu0JFu2kNl36PGBL8qRaLizglF45MEOgttlsmJc7d8jVDrsb9sTxrFFW3V7dSukbQ3Ji+iha552m98afd0XMP1AsE/y+Y4np0NvboLBVb2uAsToIrwHvaliugoQKuCqH8hK4KYJ9F3QX6B86oSkXTDn6zoaoLN2XCXPpsJsGdamQnAKVSVCfAIPx0B6nMwfYYmE2GmqjICwSNiPgQZ30qJMZdZJhf9/JpjoZlXV1sq1OuqVGnWT4GZ0sqpN8CfL5vJNp62srL51cexitvO1kwWK08hedfNXKd538tJX/7OQJ06aEJA==',
 18: b'eJwdzk0ow3Ecx/H3Pynm6d8wzMOyyEN2WeEwmq1oSy0HVlJqBy0HDvNUdpMcRIpcxIEoBwesUdp2U0ThtFxdlFpbOSkHnxxefer3+/x+3296edX/ZoevRihqhqYW6HXApROirXCtfG+Dp3awdUgXRLqlBy5c6rjBlNM+8Mt2PwQ9YBmEAS/M+2BqBEIBdUfBNQblsjsB8TDkJmFxGpYi0DAD4Vm4m4PhGCRWwBuH+zX4WYejTfjegcd9/XMIyWOInevtFdzeaOcMHDzAySuUZMH9AUOf2rWg/X5ho9ggZBrs2Q06LfzLVYCvDBYkVQkOU13ls863lPkqOJOsZHRXkFKrUpzV4KmB8VqlDV7qNaMO/gBQj0Jr',
 19: b'eJx9l1lslFUYht+/LZ1Op9Np6Ux3SlsKtGVrWUrLVtBETVBiFBSKF5C4byh4IWhUQEAFLiwXoEZxoVwAMUBQQCMmIIkRY0DAkiDVQEF2UCgoUPX98s3JOf1n9OLJ+73fds7/z8DAtsVP9T30K9DSAXz3GzCC8QrSeBLYdRxoP0HtBG7QB08BlewZexqYegaIngXqLwFLzwFrzwM7LgApl4HJV4A9zD9CLf0DWN0FTLkKrLkGHP8TmHgduPkXcILsI503gO6bQH43MOxvYD48XE31sC3NwwLPw6oUD5vo99JHAh5mBD1MIjVkXZaHmSEPbWEPK8n6bA8VvT0cyfHwe66HzDwP/aIexsU8TC3wMC3fg1fiYVmhh4IQ8HYKsDsVmOVZpgU0L7GpGW9yZk70SlrPmqjJuXMukvfXZEYw+WQ97n3cM9wZc69kd3L3JzvD3e/u9D+L+962B5OfXZVud8q5/nfpPoOo+96TvXOz2//ZCC8HEj8f/zs3/a1kPusbUq0KR9MsWelac5G8P+fOGz8+oPj3mzMktz8zcY/M+Pe66j/L7BJ1zzC98gzPZvTsM71ml/QI7t38+91zP0zyHsT/zFprnD30y/iduNpLY1HJG2963JqpCw9k2Fyrb6+7x+DuS3aOmZe9/QO2x7/X9Lh5/zlubadvT5e8B/qNaZZwIDEWnZCR2CtecGfc2TlBJVndnU9Wc5Gz3fsYLyr86DxHM3u+YdzF78iDQcX4ARmqguQM0uPmjUq/4J83c2/wz8TOkD3L7PD3mPicc+ab7M/mvmPpisTCRyGbMzyXqZj+5mBij4upy4x/t7tHSAvYOML4iyxgOnuuBSznwsDAoMZGDeLdnH9WapL7r3nD5pDWqnm3j3mHiZlWnw8pB3mPSDAR6XFjFzdn9qRFgIag7p/LfDWZEVL9Mqz6VpZqtVM3SI9gZvyzxgtb4ntayCTm52ZZ/SSseihbES81g+RcbcgFeuXY+uPkMGeWk5Ysy/Kw5UKkZ93EeQVaE9z8QnKR562LMy+ciJs/HLHx6N7q03Ntzz5qG3VhFPiKtYu8/wrm7s1T78amR3Iukuskl0gbeTem+lOOquGJqGqA92jMs3kTf0/ayS7OXcpVJDbexOXsXxSzsannc28FyWCunb4p2lMl/2RMY+ONzmJPkLOL84HLsoc+v1hjP1KTvvu4q4CzlWRMTAlG/5/dZap1ZHYJcD/3nCoCXufnXRlTLyqY3GeFGh8j7zMey54finqqydeVAk+zbxUB7/95kbKk0CJ+aoFF/EvU7aw9E6e+D/BBkfUSG3+Ne9dSl9Kf5jNsL9ZYcGOpddCfIXsrOFOs7C+xsddH/XVygH4HdXJf1TOlqsVlGndUAr/Qf1rOXvqH4yqklNl4BOdrubeEejauwt3lisnfzpmR9Kl9kyO1Wupq3r20XHmFd7gn7o2upz7K/KiK5DxEHiNT2PNqP2VzlfreAzlPv4WsYe7GAGBQNfvjXrSSuePktSpLXrXmjvQHttK/0z85S2r4/yKesXWAEnV8iLqPDKlVFW5Vq05jXyfjOwYBsVqrkhvP2lD6bkcNy+jzyYtDgDsHayw6hzpscE+u1PHzHcp/8wwD7mL/UfoNjOczd4C6kdpKne1oyXD+PjNe4LCRc9+OYr6e3/s6pWu4cp7xMWr2SGBTvTKKcWQEv5PDlekNqmXMHWzi704j69zXQT+avdfJjNH8nWjQuIy1GpLToMxtVL3F2eWkhr0tjaqLyG2Nyrwm1e4xwMwmZUU8PkltG6u8wPg90s54Jdk1TvUUyWjm92Wc5R9yeQL/LpoEfD0e+BfHC22i',
 20: b'eJx9lH1MlVUcxz8Pl4ty4XLReAmEABHuJTJDe1VLcbY0TS3NiaxGrWk1KbOWSi1bRTk1E1kryNc1rbBWtIxm015gLuaomVpjJb2ujWhJm7ZezPU9+/nscs31x2ff3+93znme3znPeb6Tez4sLsmCwZHGU9kw8Sypqi+Tjh5CYw6cyoNJ0noxs1C1C+HpXNiWZ4wvUC7tyIcVGvdEkmrvjTJ+FocLjIDGZhfDQ0WKxRLFBWeZI/4sg6UlikfDK+KvcmgphXbVfxAt4rRqoRj8KD0t3VSpWLpK+k8FLLwY2sbCuEug81JYLfaIgXHSy+B1aXSCtAoi0otEn8i8HBZfCSmTdBbToPYq2HUNnFC+eyK8I/1AbKmGvRpvlpZOVS/SyeJW1Q5Oh2fEjuth1Ezt5wZ4bDZ8MgcKZ8Ftitvnah/iVdE6D1YuhPwaOLNAvS6CO8RycadqfbUQvB1idepP3Cw23KXzE233qp+lwN3Qe496WQbP18N998Na8e0KOCIqH4a5K/WeVTCvAV56RGf0OPSs0Xue0Dk0wsuOZ7X39ep5HTy5Ub0+B+82wUfN8EYLlL2ovbTCvq3w/TbI2qlz3w1Nouc13ZM2+OUteLQdrt4LxR36fqJxn+7JftjeBTd16psfhKPd6udTqPtM+zii9xyFml44dlzn/Y2e/53W/aS7NKD78qvuxAn4+xRk/6FzOqN34dHnedQGPQZDHr+FPbxsj8/FLcPUuwcbk3Q+AShJMXU1P3djg8mJ6vDX/Z860keody/OjeF43JQUjztSE+d9HEjMZ6X8d925em7syE+Dr9T36iTjWu25LWD4NTfu49f8cVcbGjvdI8r0nM0BozM5Hvt1pyeDifOG4tZ0SXeE4LrhemZynPAw06+DiXXHcs3tCp6fAVGu8bUhy10c1bMypMd1flN0xjvTLB9acxrRvEVa97s0mmqxU58aERFTQ4YfP5BmWq18Xbr+H8WxkMXvhy1erznd8sxgpnxP9WrlKRHVw8bi9DhFmnMsYjwYNjbJf/cr35AR1wOiVb67S/kXWvOl9EBmIoOurvs3/AL9C9LUkXE/d8zPMj/fkmuevjXHvHy+dIG0U77b7Pw71/zc4fy8Slovb98u8grN0/vl6THFh6X9BXE/n1B0fj9/ocS8/E159hrFS+Th3S4uNT+vjJmfT5dPvz3G/PyQ6C03Pz8UlYdHzc9zKszP++XlM+TtddKT8u4Z0oax5ucNImO8vm+V+Xlflfl55Arz8wqn8vR/AWkSMQA=',
 21: b'eJyNmyuTXVUQhe9NJjMJCW8FVRgMfyEGQxUWjZkq7BhMfgUmAhlDFYWPx6DyExgTMx4fLHRNVs13v7v2uSOm7nnss3fvfqxe3efMb198+fX3n+12vz7c7T452+3+fbDb/bTf7X4/3+1u/j/+6//rn57d/l3ub6+9fHB7/Pbp7f3X+7vfPy52u+8e3Y27eT92ns+YPD/Hl/u7+y/Ob5/NWpknz805n32JubOOZcx8uT/XKe9cm3XfXXS5qIPMO8fz+82Tw/GzVp7Jcc4vMWfW5/XX+2M9ze/Fwzt9vDg/1Afn4J4j69wfOTI287w5P5Rt5o1erIPYML+RoemKdvGYl3jWYym/fynrar1m97n+y9P1vsZPb4pMsa334TG0r2XIccbQP2KDU89m/Ov3do9cc/26PBNdxT9pp4zZsl2L7+iA9xzDNw/ufHZ+J47++fAOM3784NYO2cPcj9+ODSIr14ptiAvZf3Qf22VfzZd8bX5n/cv3vuF98z79fc6fnx2Op3+0OLF97Av3uT/X2nX6kLGs7Wmli/ndf377+/PHh+NGP8GxOQ8+ruaJTf98due3wUjKGztmb1tyO1a4njHL/h65c40x07DH8RgZ313cyexcRmyaa7PXFlvZK/NHs5+fzTjuM/Pkuu3CnEk9cH7aJ/ufmKPMmZeyGgObnXxMPF3hXNYKllAPLU+aB2RMW4eYyTihb1j3q+NgDffZuIfn55g8b/+w77c4Jx60+F49N7/E1JXd7E8tLqlrx7/jOXtzDieGcu1VXFLe3Hv24SFX+vvxbfwRT+f+1aNjPw7Gr3CGMUmcYZzazsxPxvMce59e01iX8ZNDySmZ61dx6fzV9O442cpN3C/5D3EneXg1t2PZGNLumw/MX3KKc210/ep8e29bmGSctx5ZC3H/5mGei/hv/GhY0+LM45337Qsrrs09MB+TP7a4a5yMcgWXjQOsiyxXkzM2jixch77onMV9Wr/zd/XoMK5aXRa7MW4jo/HXzzQ8cb5qtSjXCI8mn0n8t/HEJ8rRfle199b5zD9Yaxld+8bvrTPnwZWu29q0IflW5iTfchyRB00N3fCAfMi+Sd9KXmy2jSxbucF41mJi/ojp7il4vVP+22KKfszcQa68GtPiz/nF3Nk+67iz39ru8T9j3VauIsay7+EaMZwhz12VWp0+u8VfHffkvdQDa8tVryVzhMvkGjGp+Yh5evMP4vIWV27xaI6dmsu2ab60mtd+xdhtWEEfIj6TX61qRsaP+w0tptzjcf1oDhRftZ4twwr7cy/ypzdCzp11uQ77IZ43GMY9U/4Wky2WWg3I+Zyfm95Xe7YPu66ev7dP1zVZuGHLA9yf/dhx6zGs+2P/kTHyNZ9mHWu9uU60jPYt833zN+vYNULmP9Xbdc5wj9l6t65X9nY9yD7EikO1OOCY2NrvHGxD1kDWM/vgztejQ/t5xo5eiAHm8/fh3bEH5bW/0lbmsLEPY5a47LhpOLSKI+u/+Zy5NzHbuY++HHmNQe61Nv01HThmXO+u5mp+vFV/Gi/jf/Hfq0eH+Wil25HJfSPn/MYZIyPrjpbDT3Gjxhtav8b+4Ti9bx3BPWTvtpt9l7luVT/av7dyjG3BXkKenXh3Hsw7SGJQ9sPz9BNdj785P+bE5JIrzsv9ECvZxzU+br1PnTGJUeIFn7Oc7LNscfMV97O+mk/R5+nPzpvWTeODjWt6rP3E/XLPS31zP9xr9nOFnon50FYdeIo3eV3HF/GK/dXU5MyF9CnOnzjgfO7p2B+dzz1uxW/a+dgh7+CyfuTyc+0dEmOr4VRbs9XujcMwl8z59f5QX1u198r3VnzO+cocsvGq1f4Ybxxr/HWvInqw/VnXUP6We6131/45jk6NM9YNudj8xcf5/qTl25UvRmbWEpTRuT3jn5dY4rM30N0qBsjBU0OYdzcs8B5bv9fn5rbEhVZvu4b2nKteB8cxx7QaPLZjPbqym/tj7INwTeqz1SitD9ls6L3YT7OfFrs/PDmO2/YObFV32hZbtWX21PLkKfylLtyjbT60yo3pl7X8ZM7S9JBnvX64n/lIq3eNKY2rGaMpV2qOdxeHsci16IMjE20S3ncj/bc4bXpuvtp8xfUO696Rif2h7DGyh8OmJ8B6mXthn4H1ZFubMo7u/K1DnnNdHL/Zqo/oo8TfVe3meGXtwNqfGNJiPMfX+2O82LKh1/V98iXOZdxpNXbuz3c09LHo27nS/u+Ysv97veAWv700D2IPhLnJmLbKT8EDc37afFULtXPnXmLVyh5z/PzsOP8aX5sPbvkt17ZuidPE+taj3MLtFVfiePMl8wXyplN1Y4u5yM0+SeSZ7/28F8pjrhN5XC85Bvw816YezSvYp2r9O8Zu4wXcc87D4ZyDtvbo6/5mw/kj+7pPf9R+Ew67ehdg3k/9u6fY1nV+bnzE41e6pZ7It+gH9ifqzzmv1UrU4yn/Nh48PzvWF58d345/j0x+N51n+E05v6dzTnaN7ny2wht+x0Q+yD353dyKs1An7Ztrx4Nz/you3B9r+2ixx+PwCj/jmou9ZuLMV4+P5Vnpl/f4zj/+R58wLzVGmm9MTs9a5ianYp26T5y38S3ecy/7abXqivOZFzVcpw6IK87rW/jCORxz7F+EBzm3zlr8zsO62KoRW7+sHbPOtx+67vD3W9TjqkZcxaPP42exRe6PX/h/O2jfLXwlRrX8wXk4L3tGjQ8zTxirrfMtn3a/Mmu3fbQe5vwx/69ygjkO55p+ZcsXjC3uq2Ffw3375cqHtvyhxVL71ph2ajnTcjcd8R7nYK+EOmD/KbWAex7Nbpyn1f7EqtW7cvKh+ZuYuS5zZE+rHkRb39yO11uPde77G0v7h/XpPTHWwjFaDWS9uvafMcRS16z2Jf4a6+b324+6b4y+jcPmkqs6yDq/T48nx4nHV+eHY1eczviR3onj7M35sU3b/hwrza60Fe95L+QVbe8jq2OXuNz80DnJceC+6uW+y0c72Teb/Zrv236ch/7bagfj3FYOnXvmCMnp9HFjR2zQOOwKI+0fzifGKecG9qy2asPGV6xf8yTGDOtQjmcu3sJh27HhsXlFZLTvez9NZseOY8UcpGEW55vYmWvX0hPfeZkf2nbsqbMX0XgIfdp9ocahbUfHjPP3qb5G9pbz1TvfFp/kGczzq2fcv2q4Qx9re1rxLdde1A/vsUeU8xXeTp86tT99L71Yrs3cSf07fzb9EwPMnejHxCfif9YI/28+Hh/K/0A7h/t/alzrrmzQ/Nr8pPnTyMx3+/QBP3Oq/gmvyfcDtP1/rucFeQ==',
 22: b'eJyNVztrVFEQntzcbLJxNa5P1CoKJraCjSA+KhsLQ1AQBP+Cro1lMBhQDCjaC64WWiyIW4qg1hZKsIjCNvaRqLUzHIf73W/nZFMMcx4zc2bmfDPn3vUnJ48tHhT5ovT5kMgRpaP7RZZ1/kx5c5/IitLGHpEFnT/dK/Ktneieji/r3qfDIi9mEi3q+huVe6fjaaXuLpHOTpE1Hf9Sfq0l8kD5kvLTunZ+h8itVuKzKjs/nei+rr1UflPXv6r8qd0iM02Rh0pzSn8nE839n9+eEtml9KOR6Gwz8SW1eVzXLyld1bU/jURdnb8uRb5PJO5jnDud0LUV1X08LvJR51dU9/dENTdu9Fz9eaX8jPI7RUWtRlr3OY7Xy2o8r+MbY4k2dfyoSOMP44n7nMnWTca46bnuZlnXO6fz68pXizo3mm2k+d2pas1lUH6jrPb6sHdhom4P9Y2/Hx+2aWtmD/XYJ18bBHYi+8bZ39x57G8HzvN8ROe4PcyN2WXfkNqQN88hxpfTY19N1/T8nnrgk8fHNtwvP4/X+Y757EExbMM45rpfDPtrMgdaSd/9M785XpPDe2If0E/U69F5Hr9xs+m+LkxW+xxnu6zn1mRNDu/S51wbNjfcR9jN4SFa8zvjOCNZzyXiE+PC+/SYo7iR1jSOi436ee4/1zzfW2QT5QeQ261wjjqrEOOoumD8eh1zLvlsj8vxiDawT/SC+9so6+f6WRgD42c7Nc71zDXG9Yr7UX/1fsZ7Ud44h67TL+qxsX6H7ohzk+vtuXrAWKM7tHqzuL2mMb+IddT3uu2CDscT5dExjm9LlAN/+7BWcj2V59hXc/fYI3vRfWIP4T7EZ4/C4wBi6W/xFjJ20Me3zWG7PYo3h5kcViLcOBY4R4wBrFHHToR5zA3WPsrhe8J4QX2TizDNefD8Rv6zj5gz7rORL1GvyfVGrIH1ZiXXadTfaPYP7aFfHm/07Way/tZGPcnf8AgPto7vIMcX1duovhdh2ji+70Y/p+OcIjYYT6PupNOIz47q3Ooqqm/EEGM3qlVf7xfVN2VOLsJ0u4zry+TcR8QI9xDMLfeP7bwT28E11im+hb7P+OI3x/HsOO2N1e27PNahf1Ns5SvrdCfrd5nTYaxhPWD9mewy+crfuehvhNOIPC6MD+9wVAzoKxLWTPQ2uv85POTe16h3+P1jHbutQVH3hbHF7zN/H+dqHWsy12v4e4T/kzxv+J/ke/wfFOXFdf4BjCxB8Q==',
 23: b'eJyNkz1LA0EQhp98nyR+BAtBGxOCBGwEIZUIBos0ltqIQQuxszA2aVOIQVD0HyhWgsTCPv9BECwUDgQrwbSChbucQ+Y2d2AxzO7M7PvOvTPXbZxWBh2oteHhEC734bMJhS3Y24BuAxJ1mFuFrxq8L0N+CS4WYb0K3xVTV4b7eejMwtMMNI0dGDuZhpUiXE9BeRJeCnBkrDQB3rjBy8OxB29Z2M7Bgjm3jH/NwJnxV6nA1sz9zvh2MvDVNOwmAtvMDc83BmcnEbZ+KvA9J36eHK3x/2IDg/+RGuZtvKSw65kwjpjGd/mieAX7cSzowVdYlq8V8T2CYWvtG41n+/adu873nJ7/g2t9MT3KYzVw8cVsvdZV+CSuNbM10pc991PxeGIyC43nausnw1zPXvit5GSWURiCo2M/Xlg/t97i6Zjl852aKP317HXeT8bPy+V1OW5z8fujZx23P+7/4e63nqHWwtXA5lvZUf44zTRWVD9aI/uNel97Tp1wy9l6ucubQTpea/3tvQgt4nbO9bKrvxbVr/8=',
 24: b'eJyNU8sNQjEMi8RXD3UMNmAJrtyYgBG4sgSLwCQMwAysAFZl1TJp9Q5RXt06Hyfv+Hjvz4sI2GcbcV9FXJfVA6PHHXFiOJNL7LVpGO7J0bd481y3e+LK0xxaS4/jhvgwcjM77VqOUU04o3/FXAfvMasnw8tUc1M3xoAHB7jG5bfm75lrfpj+e2DvPrMM03nqzLWObCd8fqqFajhntln/rIk7msVw/W8/u5TKK4kusKxO33Pv3XeXWo52fe5/oW9Guvfq/gKTundn',
 25: b'eJxtlnlQVVUcx79X8cFje4iIKC4hIBTmCmQoKVqRmtsgyWLmGi0maqWYpRG5BDYqaiaWaeg0qUVpjlpqVurYbqZWo0yM0Yw5BmXikjn2uXO48x7qH5/5fn/fc869557lzbu8LjkhM1oqiZQ+aiP9FW6IiZB2kKW1ltwRRh0mwFToDFmRBsePRvuiU3nWm1FSj/amXmm3kS2IaqpDaP++nfRUlNGJHaVD+J34M8xrYVvD3E7SLvQI2VvtDIkdpB/QP3jGrmgv799GTnYp1mg+z4xm/NkORm2GQXOYEmN0VDwZvoox7Rk/H78anxJjGAtJXaThnQ2t8AW0JyVK/+L341vhX4wz/MbzviJfg3a9XYpNkCLQbWS18HWCYVySFMe42V0NmXdI1+jXjXxoD2qySPzJblJ3/Pg7WX+0EH0ATnWXyiG+t7SCPhvwIfit6IoehnOwtadU3Uvy0HYRclKljqgnRaq8i7VKlkr7SCOpw2gbCGV3S3lkL9H+NP4ZfD66t6/0e5rUp7/0E/5ViLlH+rafFECWBp+mS1FkaRmcBfibLHYA+ztQ6kcdi7bNlCYNooZpkH0vewe9hkjr75MW3S/VwHr6JZNVDZOuDJb88MPwHYay/g9KqTB8uDSLLBVdM0IqhqUjpQqIyJaCslhXqB3N3NF/0PFjpDYPcWbyOKf4k2Ol6TnSgXxpMNnbcJCsepx0DM2dIL2HXzNRuvwIa4R2gvxJ0iAYNZlzAs9O4e48ihawN1P59selXyAKf/4JaV2h9Pw05gSXZvCd09lPqMJvmCm1nMUZhJTZnCk0tYhvmyO1gKXP8T3zpWVz+ZZ50toXpPRivqWEZ8B1/NGFrPMC5o8eXMy4Uu7fK9IefNgSqY66pkzavow5LmXfyqWHl3Nf0c6ruYcrpc9WsQ+vs07Uq9Zyviq4K+i8N1j/9Zx5OLeBs13JOdrIOd4kFcH2dxmzmd8T9MIWKXQra/mBlFDFGm7ju7dzriBsl1S/A3aydp/w2/Gx9PJevnMfd99mP2fioDT5EPcVPXxYeu2I9OGX0ulvJNd30n+QSVb0o/QklB9nr07Az5yNGuZfzbmFTb9KJ07zuwZXa3nXGannn7TX8XuCnoWGetb8PPt3hf1t4G5e4rzgr11lHn6Wsq5zhyxLnhaWwlyWMtyW8mFfsKWScEs7IiyNbmVpYWtLiyIt7Y6y9Hlz1sKPvbEMy5t5NdvfmzvEuUybPc7JLuDnNPNy0s/o5ua3zmyCXUbt3Ldfur/Bd5wzxvcddu3br7er6Tt93+d4h1NkX5CVo/H+Xl/uk19o0VQd7+D0s9VmkbtpX9/nObkzpgtzPUUWwrsLGbfFz2DXtjqZ3ccXp5+vtznq17T27RPKuxoaOUC2O8ioTZcAr7+xXhwojXF7x/mSwDyrXYbQAIPtpwd68/5ub7udHw3mTvqb3Onvob7YSE6g1zskuG/ObpXnUntgRpBRmwGBN2eOLwg07ZXUpcGGROrcIKNOfWNuU4bPaCS1pTSTPsdCTW37yhDpsSCjG6nzYA/tI8KNLwvxZra31a6LaT/u4bcgpClOVgH+YdIS+tc1qq9/h/8Ue+lbD5vgRJihorWp/Xn+/11aslM=',
 26: b'eJzNyj8IAXEYxvHn3F9MyipsFmVVVmWwKqtiutQpq7JJMaOsyqooi82mK6tip6yKzaPf1Y+6coPB8On7PvW6di+ziwCNKLBmH1SmIaWpHQZWZJFDU0s4UY5atDUBna2awoSuBpBlu4ZwprinQnNdulDB06G9JsW465o0o7sqJbnzNFA/uZTyUaNlyN+NSl/06agEk+CvowQzersXpP1Ykcb4f4dXm8DGBp5HZlPj',
 27: b'eJxb//mO5rMIBobgyJGDATBpK4o='})

# Render to PCM

In [ ]:
# Define a rendering algorithm that places impulses at the nearest integer sample.
# The default impulse is a single sample, but it can be redefined to implement an FIR filter during rendering.
# Uses a fixed-size buffer for no particular reason other than consistency with the engine's implementation.

class Renderer:
    def __init__(self, impulse=[1], clock_rate=4770000, sample_rate=48000, buffer_seconds=30):
        self.impulse = np.array(impulse, dtype=np.float32)
        self.sample_rate = sample_rate
        self.clocks_per_sample = clock_rate // self.sample_rate
        self.buffer = np.ones(int(buffer_seconds * self.sample_rate), dtype=np.float32)
        
    def render(self, timestamps, padding=0, phase=1):
        if hasattr(timestamps, 'timestamps'):
            timestamps = timestamps.timestamps
        if len(timestamps) < 1:
            return

        ref_sample = 0
        ref_timestamp = int(timestamps[0])

        # Clear as we go, like the real engine would
        clocks_per_sample = self.clocks_per_sample
        impulse = self.impulse
        impulse_len = len(impulse)
        buffer_len = impulse_len + padding
        self.buffer[:buffer_len].fill(0.)
        
        for timestamp in timestamps:
            # Advance quantized reference timestamp
            clocks_elapsed = (int(timestamp) - ref_timestamp) & 0xffffffff
            samples_elapsed = clocks_elapsed // clocks_per_sample
            ref_sample += samples_elapsed
            ref_timestamp = (ref_timestamp + (samples_elapsed * clocks_per_sample)) & 0xffffffff

            # Clear additional buffer space
            self.buffer[buffer_len:buffer_len+samples_elapsed].fill(0)
            buffer_len += samples_elapsed
            
            # Add/subtract impulse according to phase
            if phase < 0:
                self.buffer[ref_sample:ref_sample+impulse_len] -= impulse
                phase = 1
            else:
                self.buffer[ref_sample:ref_sample+impulse_len] += impulse
                phase = -1
        
        return self.buffer[:buffer_len]

raw = Renderer()
assert raw.render(sfx_impulse) == np.array([1.], dtype=np.float32)

In [ ]:
def audio(a, rate=48000, audio_padding=0.5, plot_padding=0.001):
    """Render audio both as a plot and as a playable / downloadable clip.
       Uses a relatively large amount of zero padding on the playable clip,
       and just a little padding on the plot so that discontinuities vs. zero
       are visible.
       """
    plt.plot(np.pad(a, int(rate * plot_padding)))
    return display.Audio(np.pad(a, int(rate * audio_padding)), rate=rate)

In [ ]:
def rendererSweep(renderer):
    """Characterize a Renderer instance against every square wave input,
       measuring the response as peak absolute value in the middle half
       of the filtered signal.
       """
    periods = np.arange(4, renderer.sample_rate/4)
    freq = renderer.sample_rate / periods
    domain = renderer.sample_rate * renderer.clocks_per_sample
    crop = renderer.sample_rate // 4
    response = np.array([ 
        np.max(np.abs(renderer.render(np.arange(0, domain, p))[crop:-crop]))
        for p in periods * renderer.clocks_per_sample
    ])
    plt.semilogy(freq, response)

# IIR Filter

In [ ]:
# This is currently my favorite design. Intentionally getting a smooth rolloff, and tuning it
# so frequencies around 2kHz are audible but not annoying. The short sound effects like soldering
# and pickup should be sharp and balanced, there shouldn't be distracting pops, and the opening
# cutscene should have a minimum of harsh tones.

class IIRFilter:
    def __init__(self,
                 sample_rate=48000,
                 padding_seconds=0.02,
                 order=2,
                 bands=(120,400),
                ):
        self.sample_rate = sample_rate
        self.padding = int(sample_rate * padding_seconds)
        self.renderer = Renderer(sample_rate=sample_rate)
        self.clocks_per_sample = self.renderer.clocks_per_sample
        self.sos = signal.butter(order, bands, 'bandpass', fs=sample_rate, output='sos')

    def render(self, timestamps):
        return signal.sosfilt(self.sos, np.pad(self.renderer.render(timestamps), (0, self.padding)))

    def plot(self):
        freq, response = signal.freqz_sos(self.sos, fs=self.sample_rate)
        plt.semilogy(0.5*self.sample_rate*freq/np.pi, np.abs(response))

iir = IIRFilter()
iir.plot()

In [ ]:
rendererSweep(iir)

In [ ]:
iir.sample_rate, iir.sos

In [ ]:
audio(iir.render(sfx_impulse))

# Sound Test

Options:
* PCM impulses, as produced by `Renderer`. Most samples are zero, with alternating single-sample impulses at each emulated speaker click. This will sound awfully harsh, and inspection of the waveform should show alternating single-sample impulses starting with the first sample and ending with the last sample. No DC bias.
* Square wave, as produced by integrating `Renderer` with `np.cumsum`. This has DC bias of about 0.5, causing clicks at the beginning and end of each effect. This is what Robot Odyssey Rewired has been producing in v0.14 and earlier.
* FIR filter design from this notebook. Works by running `Renderer` with the filter kernel as an impulse template.
* IIR filter design from this notebook. Pads the integrated impulses, and applies a series of second-order filters.

## Opening Cutscene

In [ ]:
audio(raw.render(sfx_opening_cutscene))

In [ ]:
audio(np.cumsum(raw.render(sfx_opening_cutscene)))

In [ ]:
audio(iir.render(sfx_opening_cutscene))

## Transporter

In [ ]:
audio(raw.render(sfx_transporter))

In [ ]:
audio(np.cumsum(raw.render(sfx_transporter)))

In [ ]:
audio(iir.render(sfx_transporter))

## Chip Burner

In [ ]:
audio(raw.render(sfx_chip_burner))

In [ ]:
audio(np.cumsum(raw.render(sfx_chip_burner)))

In [ ]:
audio(iir.render(sfx_chip_burner))

## 0

Player picked something up

In [ ]:
assert (raw.render(sfx_pickup) == raw.render(sound_test[0])).all()
audio(raw.render(sound_test[0]))

In [ ]:
audio(np.cumsum(raw.render(sound_test[0])))

In [ ]:
audio(iir.render(sound_test[0]))

## 1

Player dropped something

In [ ]:
audio(raw.render(sound_test[1]))

In [ ]:
audio(np.cumsum(raw.render(sound_test[1])))

In [ ]:
audio(iir.render(sound_test[1]))

## 2

Robot bumpers

In [ ]:
assert (raw.render(sfx_bump) == raw.render(sound_test[2])).all()
audio(raw.render(sound_test[2]))

In [ ]:
audio(np.cumsum(raw.render(sound_test[2])))

In [ ]:
audio(iir.render(sound_test[2]))

## 3

Robot picked something up

In [ ]:
audio(raw.render(sound_test[3]))

In [ ]:
audio(np.cumsum(raw.render(sound_test[3])))

In [ ]:
audio(iir.render(sound_test[3]))

## 5

Antenna

In [ ]:
audio(raw.render(sound_test[5]))

In [ ]:
audio(np.cumsum(raw.render(sound_test[5])))

In [ ]:
audio(iir.render(sound_test[5]))

## 6

Soldering

In [ ]:
audio(raw.render(sound_test[6]))

In [ ]:
audio(iir.render(sound_test[6]))

## 7

Power down

In [ ]:
audio(raw.render(sound_test[7]))

In [ ]:
audio(iir.render(sound_test[7]))

## 8

Power up

In [ ]:
audio(raw.render(sound_test[8]))

In [ ]:
audio(iir.render(sound_test[8]))

## 13

Shortest effect?

In [ ]:
audio(raw.render(sound_test[13]))

In [ ]:
audio(iir.render(sound_test[13]))

## 19

In [ ]:
audio(raw.render(sound_test[19]))

In [ ]:
audio(iir.render(sound_test[19]))

## 21

In [ ]:
audio(raw.render(sound_test[21]))

In [ ]:
audio(iir.render(sound_test[21]))

## 23

In [ ]:
audio(raw.render(sound_test[23]))

In [ ]:
audio(iir.render(sound_test[23]))

## 27

In [ ]:
audio(raw.render(sound_test[27]))

In [ ]:
audio(iir.render(sound_test[27]))